## Part 1. Declare needed Python packages (done)

In [1]:
#packages for manipulating data
import pandas as pd
import numpy as np

#packages for visualizing data
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
# import graphviz
# import pydotplus
import io
from scipy import misc


#packages for preprocessing data
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

#packages for data mining
from sklearn.neural_network import MLPClassifier

#package for evaluate datamining model in term of accuracy
from sklearn.metrics import accuracy_score



#package for connecting to SQL Server
import pyodbc

#setting up environment
%matplotlib inline
pd.set_option('display.max_columns',500 )
import warnings
warnings.filterwarnings('ignore')   #turn off warning

### 6. Read csv data to dataframe

In [2]:
data = pd.read_csv(r'C:\users\vdavis\numeric_vTargetMail_ICA04a.csv')
data.head()

,Unnamed: 0,MaritalStatus_Num,Gender_Num,YearlyIncome,TotalChildren,NumberChildrenAtHome,EnglishEducation_Num,EnglishOccupation_Num,HouseOwnerFlag,NumberCarsOwned,Age,BikeBuyer
0,0,1,1,20000.0,0,0,3,2,0,0,36,1
1,1,0,1,20000.0,0,0,3,2,1,0,37,1
2,2,1,1,10000.0,0,0,1,0,0,2,39,1
3,3,1,0,10000.0,1,1,1,0,0,0,41,1
4,4,1,0,10000.0,1,1,1,0,0,0,38,1


### 7. Training and testing
How do you know how well a model will generalize to new cases? A common option is to split your data into two sets: the training set and the test set. As these names imply, you train your model using the training set, and you test it using the test set. The error rate on new cases is called the generalization error (or out-of-sample error), and by evaluating your model on the test set, you get an estimation of this error. This value tells you how well your model will perform on instances it has never seen before.

The SciKit library provides a tool, called the Model Selection library. There’s a class in the library which is, aptly, named ‘train_test_split.’

test_size — This parameter decides the size of the data that has to be split as the test dataset. This is given as a fraction. For example, if you pass 0.5 as the value, the dataset will be split 50% as the test dataset. If you’re specifying this parameter, you can ignore the next parameter.
train_size — You have to specify this parameter only if you’re not specifying the test_size. This is the same as test_size, but instead you tell the class what percent of the dataset you want to split as the training set.
random_state — Here you pass an integer, which will act as the seed for the random number generator during the split. Or, you can also pass an instance of the RandomState class, which will become the number generator. If you don’t pass anything, the RandomState instance used by np.random will be used instead.
We split our dataset (data) into two different datasets training and test.

#### a) Split data
We split our dataset (data) into two different datasets: training and test.

In [3]:
from sklearn.model_selection import train_test_split
training, test = train_test_split(data, test_size =1000)
print("Training size: {}; Test size:{}".format(len(training), len(test)))

Training size: 17484; Test size:1000


#### b) Determine independent features and the target feature (dependent variable) in training set and test set.

In [4]:
features = ['MaritalStatus_Num', 'Gender_Num', 'YearlyIncome', 'TotalChildren',
       'NumberChildrenAtHome', 'EnglishEducation_Num', 'EnglishOccupation_Num',
       'HouseOwnerFlag', 'NumberCarsOwned', 'Age']
x_train = training[features]
y_train = training["BikeBuyer"]
x_test = test[features]
y_test = test["BikeBuyer"]

### 8. Multi-layer perceptron in Sklearn</b>

<i>https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html</i>

Class MLPClassifier implements a multi-layer perceptron (MLP) algorithm that trains using Back propagation.

<b><i>Syntax:</i></b>

<i>class sklearn.neural_network.MLPClassifier(hidden_layer_sizes=(100, ), activation=’relu’, solver=’adam’, alpha=0.0001, batch_size=’auto’, learning_rate=’constant’, learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10)</i>

<b><i>Parameters:</i></b>

<br>
<i>hidden_layer_sizes : tuple, length = n_layers - 2, default (100,)</i>

The ith element represents the number of neurons in the ith hidden layer.

<br>
<i>activation : {‘identity’, ‘logistic’, ‘tanh’, ‘relu’}, default ‘relu’</i>

Activation function for the hidden layer.

‘identity’, no-op activation, useful to implement linear bottleneck, returns f(x) = x
‘logistic’, the logistic sigmoid function, returns f(x) = 1 / (1 + exp(-x)).
‘tanh’, the hyperbolic tan function, returns f(x) = tanh(x).
‘relu’, the rectified linear unit function, returns f(x) = max(0, x)

<br>
<i>solver : {‘lbfgs’, ‘sgd’, ‘adam’}, default ‘adam’</i>

The solver for weight optimization.

‘lbfgs’ is an optimizer in the family of quasi-Newton methods.
‘sgd’ refers to stochastic gradient descent.
‘adam’ refers to a stochastic gradient-based optimizer proposed by Kingma, Diederik, and Jimmy Ba

<br>
<i>batch_size : int, optional, default ‘auto’</i>

Size of minibatches for stochastic optimizers. If the solver is ‘lbfgs’, the classifier will not use minibatch. When set to “auto”, batch_size=min(200, n_samples)

<br>
<i>learning_rate : {‘constant’, ‘invscaling’, ‘adaptive’}, default ‘constant’</i>
Learning rate schedule for weight updates.

<li>‘constant’ is a constant learning rate given by ‘learning_rate_init’.
<li>‘invscaling’ gradually decreases the learning rate at each time step ‘t’ using an inverse scaling exponent of ‘power_t’. effective_learning_rate = learning_rate_init / pow(t, power_t)
<li>‘adaptive’ keeps the learning rate constant to ‘learning_rate_init’ as long as training loss keeps decreasing. Each time two consecutive epochs fail to decrease training loss by at least tol, or fail to increase validation score by at least tol <i>if ‘early_stopping’ is on, the current learning rate is divided by 5.
Only used when solver='sgd'

<br>
<i>max_iter : int, optional, default 200</i>

Maximum number of iterations. The solver iterates until convergence (determined by ‘tol’) or this number of iterations. For stochastic solvers (‘sgd’, ‘adam’), note that this determines the number of epochs (how many times each data point will be used), not the number of gradient steps.

<br>
<i>random_state : int, RandomState instance or None, optional, default None</i>

If int, random_state is the seed used by the random number generator; If RandomState instance, random_state is the random number generator; If None, the random number generator is the RandomState instance used by np.random.

#### 8.1. Using MLP with unstandardized dataset


##### a) Define a predictive model using MLP classifier

In [5]:
model = MLPClassifier(solver='lbfgs',hidden_layer_sizes=(15, 2), max_iter  = 20, random_state=1)

##### b) Train the model

In [6]:
dt = model.fit(x_train ,y_train)

##### b) Make prediction and evaluate the accuracy of the model on test set

In [7]:
predictions = model.predict(x_test)
print("Accuracy:",accuracy_score(y_test,predictions))

Accuracy: 0.511


In [8]:
from sklearn.metrics import confusion_matrix
print("Confusion matrix:")
confusion_matrix(y_test, predictions)

Confusion matrix:


array([[511,   0],
       [489,   0]], dtype=int64)

### 9. Rescale/Standardize data

When your data is comprised of attributes with varying scales, many machine learning algorithms can benefit from rescaling the attributes to all have the same scale.

Often this is referred to as normalization and attributes are often rescaled into the range between 0 and 1. This is useful for optimization algorithms in used in the core of machine learning algorithms like gradient descent. It is also useful for algorithms that weight inputs like regression and neural networks and algorithms that use distance measures like K-Nearest Neighbors. (Jason Brownlee)

Standardization of datasets is a common requirement for many machine learning estimators implemented in scikit-learn; they might behave badly if the individual features do not more or less look like standard normally distributed data: Gaussian with zero mean and unit variance.

In practice we often ignore the shape of the distribution and just transform the data to center it by removing the mean value of each feature, then scale it by dividing non-constant features by their standard deviation.

The sklearn.preprocessing package provides a utility classes such as MinMaxScaler, StandardScaler that implement the Transformer API to compute the mean and standard deviation on a training set so as to be able to later reapply the same transformation on the testing set.

In the below cell switch between MinMaxScaler and StandardScaler to see how well the MLP performs prediction

In [9]:
sc = StandardScaler()
# sc = MinMaxScaler(feature_range=(0, 1))
sc.fit(x_train)
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

#### a) Train a model on standardized dataset

In [10]:
std_model = MLPClassifier(solver='lbfgs',hidden_layer_sizes=(15, 2), max_iter  = 20, random_state=1)
sdt = std_model.fit(x_train_std ,y_train)

##### b) Make prediction and evaluate on test set

In [11]:
predictions = std_model.predict(x_test_std)
print(accuracy_score(y_test,predictions))

0.691


Comment: The MLP made slightly better prediction on standardized data than 0-1 rescaled data

# 10. Change the parameters of MLP 

We can increase the number of neurons in the hidden layers/ number on iteration to see if the network be able to improve the prediction.

In [12]:
advanced_model = MLPClassifier(solver='lbfgs',hidden_layer_sizes=(150, 3), max_iter  = 25, random_state=1)
sdt = advanced_model.fit(x_train_std ,y_train)
predictions = std_model.predict(x_test_std)
print(accuracy_score(y_test,predictions))

0.691
